# Set up environments

## Import necessary packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pathlib
import os
import glob
import json

from helpers.Vehicle import Vehicle
from helpers.Gaze import Gaze

pd.options.display.float_format = '{:}'.format

## Load eye-tracking data

In [ ]:
i = 4
exp_dir = pathlib.Path(".").joinpath(f"run_{i}").absolute()
gaze_dir = max(glob.glob(os.path.join(exp_dir, '*/')), key=os.path.getmtime)
export_dir = pathlib.Path(gaze_dir)

# Load in dataframe
raw_gaze_positions_df = pd.read_csv(export_dir.joinpath("gaze_positions.csv"))
raw_gaze_positions_df[["gaze_timestamp", "norm_pos_x", "norm_pos_y"]].head(10)

# TODO - Figure out why it is not monotonically increasing
# assert raw_gaze_positions_df["gaze_timestamp"].is_monotonic_increasing

In [2]:
RUN_IDX = 4 # e.g., run_{RUN_IDX}

SAVE_DIR = pathlib.Path(".").joinpath(f"run_{RUN_IDX}").absolute()
GAZE_DIR = pathlib.Path(max(glob.glob(os.path.join(SAVE_DIR, '*/')), key=os.path.getmtime))

PATH_GAZE = GAZE_DIR.joinpath("gaze_positions.csv")
PATH_BLINK = GAZE_DIR.joinpath("blinks.csv")

PATH_VEHICLE = SAVE_DIR.joinpath("df_vehicle.csv")

print(PATH_GAZE)
print(PATH_VEHICLE)

gaze = Gaze(PATH_GAZE, PATH_BLINK)
vehicle = Vehicle(PATH_VEHICLE)

gaze.get_df().head(10)

/Users/jhpark/Downloads/BAAR/run_4/000/gaze_positions.csv
/Users/jhpark/Downloads/BAAR/run_4/df_vehicle.csv


,gaze_timestamp,confidence,norm_pos_x,norm_pos_y,gaze_point_3d_x,gaze_point_3d_y,gaze_point_3d_z
0,0.009873000002698973,0.3772007366575282,0.3968546633786421,0.1903110676833806,-70.90863464018116,105.3522480109178,433.6360474436174
1,0.02328199995099567,0.4196345079846225,0.3177064469429876,0.2464397433430175,-130.71814910367476,83.92418778650772,433.7237894567565
2,0.0338929999852553,0.3893545317863013,0.3917971707676194,0.1698380240992133,-74.50110949984386,113.59788274468396,430.6418751285349
3,0.04437600000528619,0.3979829574542226,0.2959662548101496,0.3076271152577555,-148.0976061576245,58.97884239959882,436.6767745991757
4,0.05531100000371225,0.4049523977299577,0.378967299900278,0.2633954506392396,-84.62849390737738,76.14763649452073,441.07176390983057
5,0.06865000000107102,0.3890681760767794,0.4062456042890149,0.3021113938830071,-64.55394024763046,60.09465599036571,445.7510173196515
6,0.07730000000447035,0.4049190816612556,0.3275219637943097,0.2982842705929435,-123.76483823918636,62.50127975418189,440.1236837474422
7,0.01445299998158589,0.9369034171104432,0.6558109594928754,0.7047228236217133,115.67135641350934,-103.6548404318408,419.35899820281503
8,0.005260000005364418,0.9448680877685548,0.6758415407094792,0.7625591359665012,132.16932546313595,-129.1016097790039,416.0224596485724
9,0.08921099998406135,0.4095073296190545,0.3270591495792997,0.2381522859285092,-123.514495936027,87.16922464356003,433.9596621169718


### Remove section before start

In [ ]:
ts_blink = None # timestamp when blink ended
blink_df = pd.read_csv(export_dir.joinpath("blinks.csv"))
#select end frame index from the first instance of a blink longer than 0.5s
ts_blink = blink_df[blink_df['duration'] > 0.5].iloc[0]['end_timestamp']

# Shift Pupil timestamp to start at t = 0
#removing data before the blink timestamp
gaze_positions_df = raw_gaze_positions_df[raw_gaze_positions_df['gaze_timestamp'] >= ts_blink].reset_index(drop=True)
#timestamp of the first entry in the datafram used to offset the other timestamps
time_bias = gaze_positions_df.iloc[0]['gaze_timestamp']
print(f"Timestamp of blank: {time_bias}")
gaze_positions_df['gaze_timestamp'] -= time_bias

gaze_positions_df = gaze_positions_df[gaze_positions_df['gaze_timestamp'] > 0.0].reset_index(drop=True)

gaze_positions_df[["gaze_timestamp", "norm_pos_x", "norm_pos_y"]].head(10)

In [ ]:
# Filter out of bound datapoints
gaze_positions_df = gaze_positions_df[(gaze_positions_df['norm_pos_x'] <= 1.0) & (gaze_positions_df['norm_pos_x'] >= 0.0)].reset_index(drop=True)
gaze_positions_df = gaze_positions_df[(gaze_positions_df['norm_pos_y'] <= 1.0) & (gaze_positions_df['norm_pos_y'] >= 0.0)].reset_index(drop=True)

In [ ]:
gaze_positions_df.columns
"""
# Initially, relative to the start of the recording. Then, normalized to the start of the video.
    'gaze_timestamp', 'world_index', 'confidence', 

# Normalized gaze position in the eye camera image. 
    'norm_pos_x', 'norm_pos_y', 
    'base_data', 

# 3D gaze point in world coordinates.   
    'gaze_point_3d_x', 'gaze_point_3d_y', 'gaze_point_3d_z', 

# 3D gaze vector in world coordinates.
    'eye_center0_3d_x', 'eye_center0_3d_y', 'eye_center0_3d_z', 
    'gaze_normal0_x', 'gaze_normal0_y', 'gaze_normal0_z', 
    'eye_center1_3d_x', 'eye_center1_3d_y', 'eye_center1_3d_z', 
    'gaze_normal1_x', 'gaze_normal1_y', 'gaze_normal1_z', 
    
    'gaze_timestamp_unix', 'gaze_timestamp_datetime', 'video_timestamp', 'video_timestamp_truncated'
"""

In [ ]:
gaze_positions_df[["gaze_timestamp", "norm_pos_x", "norm_pos_y"]].head(10)

## Examine gaze data and prase into N laps

In [ ]:
print(gaze_positions_df['norm_pos_x'].describe())
print()
print(gaze_positions_df['norm_pos_y'].describe())

<span style="color:red">Q. Why are normalized positions out of [0, 1]?</span> \
Check out the [documentation](https://docs.pupil-labs.com/core/terminology/#coordinate-system) \
<span style="color:green">R. 1. Maybe just some outliers. -> remove them </span>

In [ ]:
num_gaze_timestamps = len(gaze_positions_df['gaze_timestamp'])
print(num_gaze_timestamps)

In [ ]:
plt.figure(figsize=(14, 6))

# Plot norm_pos_x
plt.scatter(gaze_positions_df['gaze_timestamp'], gaze_positions_df['norm_pos_x'])
plt.xlabel('gaze_timestamp')
plt.ylabel('norm_pos_x')
plt.title('Scatter Plot of norm_pos_x')
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))

# Plot norm_pos_y
plt.scatter(gaze_positions_df['gaze_timestamp'], gaze_positions_df['norm_pos_y'])
plt.xlabel('gaze_timestamp')
plt.ylabel('norm_pos_y')
plt.title('Scatter Plot of norm_pos_y')
plt.show()

In [ ]:
# Plot norm_pos_x and norm_pos_y
plt.scatter(gaze_positions_df['norm_pos_x'], gaze_positions_df['norm_pos_y'])
plt.xlabel('norm_pos_x')
plt.ylabel('norm_pos_y')
plt.title('Scatter Plot of norm_pos_x and norm_pos_y')
plt.show()

In [ ]:
# Plot histogram of norm_pos_x
plt.hist(gaze_positions_df['norm_pos_x'], bins=20, alpha=0.5, color='blue', label='norm_pos_x')

# Plot histogram of norm_pos_y
plt.hist(gaze_positions_df['norm_pos_y'], bins=20, alpha=0.5, color='red', label='norm_pos_y')

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of norm_pos_x and norm_pos_y')

# Add legend
plt.legend()

# Show the plot
plt.show()

## Load vehicle data

In [ ]:
# load in processed vehicle data stored in .csv format
df_vehicle = pd.read_csv('df_vehicle.csv')
df_vehicle = df_vehicle.iloc[1:] # remove first row - not necessary datapoint
df_vehicle.describe()

print("Number of columns in the dataframe: ", len(df_vehicle.columns), "\n")
print(df_vehicle.columns)

"""
# Time
    'time_sim'
    'time_real'

# Pedal
    'Pos_AccPedal___', 'Pos_BrakePedal___', 'Pos_ClutchPedal___'

# Ego Position & Dynamics
    'Pos_x_Vehicle_CoorSys_E_m_',
    'Pos_y_Vehicle_CoorSys_E_m_',
    'Pos_z_Vehicle_CoorSys_E_m_',

    'Angle_Roll_Vehicle_CoorSys_E_deg_',
    'Angle_Pitch_Vehicle_CoorSys_E_deg_',
    'Angle_Yaw_Vehicle_CoorSys_E_deg_', 

    'v_x_Vehicle_CoG_km_h_',
    'v_y_Vehicle_CoG_km_h_', '
    'v_z_Vehicle_CoG_km_h_',
    'v_Total_Vehicle_km_h_', 

    'YawRate_Vehicle_CoG_deg_s_', 
    's_Veh_Road_m_',
    'd_Total_Veh_Road_m_'
"""

### Data proprocessing (Renaming & Removing unnecessary)

In [ ]:
# Rename
df_vehicle = df_vehicle.rename(columns={"Pos_x_Vehicle_CoorSys_E_m_": "ego_x", 
                                        "Pos_y_Vehicle_CoorSys_E_m_": "ego_y",
                                        "Pos_z_Vehicle_CoorSys_E_m_": "ego_z",
                                        "Pos_BrakePedal___": "Brake[Bar]_avg",                       # TODO - FIX
                                        "Angle_Yaw_Vehicle_CoorSys_E_deg_": "a_y_Vehicle_CoG[m|s2]", # TODO - FIX
                                        "YawRate_Vehicle_CoG_deg_s_": "YawRate_Vehicle_CoG[deg|s]",
                                        "v_x_Vehicle_CoG_km_h_": "v_x_Vehicle_CoG[km|h]"
                                        })

In [ ]:
# Remove data before starting
df_vehicle = df_vehicle[(df_vehicle['Brake[Bar]_avg'] == 0.0) & (df_vehicle.index > 1000)] # Use break-off signal to discard section before
df_vehicle = df_vehicle.reset_index(drop=True)
df_vehicle.loc[:, 'time_real'] = df_vehicle['time_real'] - df_vehicle['time_real'].iloc[0]

# Add distance column with distance values in each timestep

# Compute the difference in each time step
df_vehicle['ego_x_diff'] = df_vehicle['ego_x'].diff()
df_vehicle['ego_y_diff'] = df_vehicle['ego_y'].diff()
df_vehicle[['ego_x_diff', 'ego_y_diff']] = df_vehicle[['ego_x_diff', 'ego_y_diff']].fillna(0)

df_vehicle['delta_s'] = np.sqrt(df_vehicle['ego_x_diff']**2 + df_vehicle['ego_y_diff']**2) # distance between two timestep
df_vehicle['s'] = df_vehicle['delta_s'].cumsum()

df_vehicle[['ego_x', 'ego_y', 'ego_x_diff', 'ego_y_diff', 'delta_s', 's']].head(10)

In [ ]:
n_laps = 10

def find_veh_lap_end_idx(df):
    ''' Find index in df_vehicle where the vehicle complese a single lap'''

    df.loc[:, 'ego_y_offset'] = df['ego_y'] - df['ego_y'].iloc[0]
    sign_changes = np.where((df['ego_y_offset'].shift(1) < 0) & (df['ego_y_offset'] >= 0))[0]

    return pd.Index(sign_changes)

lap_end_idx = find_veh_lap_end_idx(df_vehicle)
print(lap_end_idx)

veh_lap_end_ts = np.array(df_vehicle.loc[lap_end_idx]["time_real"])

def find_gaze_lap_end_idx(gaze_positions_df, veh_lap_end_ts):
    '''Find index of the closes timestep in df_gaze for each index '''
    gaze_lap_end_idx = []
    for ts in veh_lap_end_ts:
        gaze_lap_end_idx.append((np.abs(gaze_positions_df['gaze_timestamp'] - ts)).idxmin())
    return pd.Index(gaze_lap_end_idx)

gaze_lap_end_idx = find_gaze_lap_end_idx(gaze_positions_df, veh_lap_end_ts)

print(gaze_lap_end_idx)

assert len(lap_end_idx) == n_laps
assert len(gaze_lap_end_idx) == n_laps

### Visualize trajectory of the vehicle

In [ ]:
# plt.plot(df_vehicle['ego_x'], df_vehicle['ego_y'])
# # plt.plot(df_vehicle['ego_x'][:10000], df_vehicle['ego_y'][:10000], 'dodgerblue', marker='.') 

# start_pos = df_vehicle.iloc[0][['ego_x', 'ego_y']]
# start_pos_x, start_pos_y = start_pos['ego_x'], start_pos['ego_y']
# plt.plot([start_pos_x-5.0, start_pos_x+5.0], [start_pos_y, start_pos_y], 'k', marker='o') # starting line

# plt.xlabel('Ego X')
# plt.ylabel('Ego Y')
# plt.title('Ego Position')
# plt.show()

### Plot Attention time vs. # of laps

In [ ]:
# from sklearn.linear_model import LinearRegression
# laps = np.arange(1,11,1)
# attention_time = np.array([8.94,6.15,4.58,4.16,3.34,2.97,2.76,2.39,1.56,1.44]) # Need to be replaced with more consistent measure
# inv_x = 1/laps
# reg = LinearRegression().fit(inv_x[:, None], attention_time[:, None])

# print("Coefficient of determination:", reg.score(inv_x[:, None], attention_time[:, None]))
# print("intercept:", reg.intercept_)
# print("slope:", reg.coef_)      

# import matplotlib.pyplot as plt

# plt.figure()
# plt.scatter(laps, attention_time)
# plt.grid()
# plt.xlim([1,10])
# plt.xlabel('Lap #')
# plt.ylabel('Attention/fixation time to intersection stop sign [s]')
# plt.title('Attention variation over laps')

### Visualize figures

In [ ]:
def get_intervals(idx):
    idx_lst = []
    for i in range(len(idx)):
        if i == 0:
            idx_lst.append((0, idx[i]))
        else:   
            idx_lst.append((idx[i-1]+1, idx[i]))

    return idx_lst

lap_idx = get_intervals(lap_end_idx)
gaze_lap_idx = get_intervals(gaze_lap_end_idx)

In [ ]:
column_labels = ['Brake[Bar]_avg', 'a_y_Vehicle_CoG[m|s2]', 'YawRate_Vehicle_CoG[deg|s]', 'v_x_Vehicle_CoG[km|h]']

In [ ]:
def find_stop_point(df_vehicle, start_idx, end_idx, e=0.1):
    """
    This function finds stoppoing point of the vehicle and returns the corresponding norm_idx.
        Stopping point is defined when the vehicle speed is less than 0.1 km/h.

    Returns:
        norm_idx: The result of the operation.
    """
    total_interval = end_idx - start_idx
    window_start_idx = start_idx + int(total_interval * 0.25)
    window_end_idx   = start_idx + int(total_interval * 0.75)
    norm_idx = df_vehicle['v_x_Vehicle_CoG[km|h]'][window_start_idx:window_end_idx].lt(e).idxmax()
    
    return norm_idx

def section_idx(start_idx, end_idx):
    total_interval = end_idx - start_idx
    start_idx = start_idx + int(total_interval * 0.50)
    end_idx = end_idx - int(total_interval * 0.20)
    return start_idx, end_idx

## Distance domain normalization

In [ ]:
num_laps = 10
fig, axs = plt.subplots(num_laps, len(column_labels), figsize=(20, 20))

for i in range(num_laps):
    start_idx, end_idx = lap_idx[i][0], lap_idx[i][1]
    
    gaze_start_idx, gaze_end_idx = gaze_lap_idx[i][0], gaze_lap_idx[i][1]

    norm_idx = find_stop_point(df_vehicle, start_idx, end_idx, e=0.75)
    gaze_norm_idx = norm_idx // 4

    start_idx, end_idx = section_idx(start_idx, end_idx)

    x = df_vehicle['s'][start_idx:end_idx] - df_vehicle['s'][norm_idx]

    print(f"Lap {i+1}: {df_vehicle['s'][norm_idx]}")

    # Column data
    brake_pressure = df_vehicle['Brake[Bar]_avg'][start_idx:end_idx]
    v_x            = df_vehicle['v_x_Vehicle_CoG[km|h]'][start_idx:end_idx]
    a_y            = df_vehicle['a_y_Vehicle_CoG[m|s2]'][start_idx:end_idx]
    yaw            = df_vehicle['YawRate_Vehicle_CoG[deg|s]'][start_idx:end_idx]
    vline          = 0 # df_vehicle['x'][norm_idx]

    # Plot brake pressure
    axs[i, 0].plot(x, brake_pressure, label=f'Lap {i+1}')
    axs[i, 0].set_title(f'Lap {i+1} - stop sign interval braking behavior')
    axs[i, 0].axvline(x=vline, color='red', linestyle='--')
  
    # Plot velocity
    axs[i, 1].plot(x, v_x, label=f'Lap {i+1}')
    axs[i, 1].set_title(f'Lap {i+1} - X-direction velocity of Vehicle CoG')
    axs[i, 1].axvline(x=vline, color='red', linestyle='--')

    # Plot acceleration
    axs[i, 2].plot(x, a_y, label=f'Lap {i+1}')
    axs[i, 2].set_title(f'Lap {i+1} - Y-direction acceleration of Vehicle CoG')
    axs[i, 2].axvline(x=vline, color='red', linestyle='--')

    #Plot Yaw rate
    axs[i, 3].plot(x, yaw, label=f'Lap {i+1}')
    axs[i, 3].set_title(f'Lap {i+1} - Yaw rate of Vehicle CoG')
    axs[i, 3].axvline(x=vline, color='red', linestyle='--')

    # Set common labels
    for j in range(len(column_labels)):
        axs[i, j].set_xlabel('x (m)')

axs[5, 0].set_ylabel('Mean Brake Pressure Across 4 Wheels [Bar]', fontsize = 16)
axs[5, 1].set_ylabel('center of gravity x-velocity [Km/h]', fontsize = 16)
axs[5, 2].set_ylabel('center of gravity y-acceleration [m/(s^2)]', fontsize = 16)
axs[5, 3].set_ylabel('center of gravity yaw rate [deg/s]', fontsize = 16)

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

## Time domain normalization

In [ ]:
num_laps = 10
fig, axs = plt.subplots(num_laps, len(column_labels), figsize=(20, 20))

for i in range(num_laps):
    
    start_idx, end_idx = lap_idx[i][0], lap_idx[i][1]
    gaze_start_idx, gaze_end_idx = gaze_lap_idx[i][0], gaze_lap_idx[i][1]

    norm_idx = find_stop_point(df_vehicle, start_idx, end_idx, e=0.75)
    
    gaze_norm_idx = norm_idx // 4

    time = df_vehicle['time_real'][start_idx:end_idx] - df_vehicle['time_real'][norm_idx]
    
    gaze_start_idx, gaze_end_idx = section_idx(gaze_start_idx, gaze_end_idx)
    gaze_time = gaze_positions_df['gaze_timestamp'][gaze_start_idx:gaze_end_idx] - gaze_positions_df['gaze_timestamp'][gaze_norm_idx]

    # Column data
    brake_pressure = df_vehicle['Brake[Bar]_avg'][start_idx:end_idx]
    v_x            = df_vehicle['v_x_Vehicle_CoG[km|h]'][start_idx:end_idx]
    a_y            = df_vehicle['a_y_Vehicle_CoG[m|s2]'][start_idx:end_idx]
    yaw            = df_vehicle['YawRate_Vehicle_CoG[deg|s]'][start_idx:end_idx]
    vline          = 0 # df_vehicle['time'][norm_idx]

    # Plot variables
    YTICKS = [0.25, 0.375, 0.5, 0.625, 0.75] 

    # Plot brake pressure
    axs[i, 0].plot(time, brake_pressure, label=f'Lap {i+1}')
    axs[i, 0].set_title(f'Lap {i+1} - stop sign interval braking behavior')
    axs[i, 0].axvline(x=vline, color='red', linestyle='--')
    
    ax2 = axs[i, 0].twinx()
    norm_pos_x = gaze_positions_df['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', alpha=0.5)
    ax2.set_ylabel('Gaze norm_pos_x')
  
    # Plot velocity
    axs[i, 1].plot(time, v_x, label=f'Lap {i+1}')
    axs[i, 1].set_title(f'Lap {i+1} - X-direction velocity of Vehicle CoG')
    axs[i, 1].axvline(x=vline, color='red', linestyle='--')
    ax2 = axs[i, 1].twinx()
    norm_pos_x = gaze_positions_df['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', alpha=0.5)
    ax2.set_ylabel('Gaze norm_pos_x')

    # Plot acceleration
    axs[i, 2].plot(time, a_y, label=f'Lap {i+1}')
    axs[i, 2].set_title(f'Lap {i+1} - Y-direction acceleration of Vehicle CoG')
    axs[i, 2].axvline(x=vline, color='red', linestyle='--')
    ax2 = axs[i, 2].twinx()
    norm_pos_x = gaze_positions_df['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', alpha=0.5)
    ax2.set_ylabel('Gaze norm_pos_x')

    #Plot Yaw rate
    axs[i, 3].plot(time, yaw, label=f'Lap {i+1}')
    axs[i, 3].set_title(f'Lap {i+1} - Yaw rate of Vehicle CoG')
    axs[i, 3].axvline(x=vline, color='red', linestyle='--')
    ax2 = axs[i, 3].twinx()
    norm_pos_x = gaze_positions_df['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', alpha=0.5)
    ax2.set_ylabel('Gaze norm_pos_x')
    

    # Set common labels
    for j in range(len(column_labels)):
        axs[i, j].set_xlabel('Time (s)')

axs[5, 0].set_ylabel('Mean Brake Pressure Across 4 Wheels [Bar]', fontsize = 16)
axs[5, 1].set_ylabel('center of gravity x-velocity [Km/h]', fontsize = 16)
axs[5, 2].set_ylabel('center of gravity y-acceleration [m/(s^2)]', fontsize = 16)
axs[5, 3].set_ylabel('center of gravity yaw rate [deg/s]', fontsize = 16)

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
num_laps = 10
fig, axs = plt.subplots(num_laps, len(column_labels), figsize=(20, 20))

for i in range(num_laps):
    
    start_idx, end_idx = lap_idx[i][0], lap_idx[i][1]
    gaze_start_idx, gaze_end_idx = gaze_lap_idx[i][0], gaze_lap_idx[i][1]

    norm_idx = find_stop_point(df_vehicle, start_idx, end_idx, e=0.75)
    
    gaze_norm_idx = norm_idx // 4

    time = df_vehicle['time'][start_idx:end_idx] - df_vehicle['time'][norm_idx]
    
    gaze_time = gaze_positions_df['gaze_timestamp'][gaze_start_idx:gaze_end_idx] - gaze_positions_df['gaze_timestamp'][gaze_norm_idx]

    # Column data
    brake_pressure = df_vehicle['Brake[Bar]_avg'][start_idx:end_idx]
    v_x            = df_vehicle['v_x_Vehicle_CoG[km|h]'][start_idx:end_idx]
    a_y            = df_vehicle['a_y_Vehicle_CoG[m|s2]'][start_idx:end_idx]
    yaw            = df_vehicle['YawRate_Vehicle_CoG[deg|s]'][start_idx:end_idx]
    vline          = 0 # df_vehicle['time'][norm_idx]

    # Plot variables
    YTICKS = [0.25, 0.375, 0.5, 0.625, 0.75] 

    # Plot brake pressure
    axs[i, 0].plot(time, brake_pressure, label=f'Lap {i+1}')
    axs[i, 0].set_title(f'Lap {i+1} - stop sign interval braking behavior')
    axs[i, 0].axvline(x=vline, color='red', linestyle='--')
    
    ax2 = axs[i, 0].twinx()
    norm_pos_x = gaze_positions_df['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', alpha=0.5)
    ax2.set_ylabel('Gaze norm_pos_x')
    ax2.set_ylim(0.25, 0.75)
    # ax2.set_yticks(YTICKS)
  
    # Plot velocity
    axs[i, 1].plot(time, v_x, label=f'Lap {i+1}')
    axs[i, 1].set_title(f'Lap {i+1} - X-direction velocity of Vehicle CoG')
    axs[i, 1].axvline(x=vline, color='red', linestyle='--')
    ax2 = axs[i, 1].twinx()
    norm_pos_x = gaze_positions_df['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', alpha=0.5)
    ax2.set_ylabel('Gaze norm_pos_x')
    ax2.set_ylim(0.25, 0.75)
    # ax2.set_yticks(YTICKS)

    # Plot acceleration
    axs[i, 2].plot(time, a_y, label=f'Lap {i+1}')
    axs[i, 2].set_title(f'Lap {i+1} - Y-direction acceleration of Vehicle CoG')
    axs[i, 2].axvline(x=vline, color='red', linestyle='--')
    ax2 = axs[i, 2].twinx()
    norm_pos_x = gaze_positions_df['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', alpha=0.5)
    ax2.set_ylabel('Gaze norm_pos_x')
    ax2.set_ylim(0.25, 0.75)
    # ax2.set_yticks(YTICKS)

    #Plot Yaw rate
    axs[i, 3].plot(time, yaw, label=f'Lap {i+1}')
    axs[i, 3].set_title(f'Lap {i+1} - Yaw rate of Vehicle CoG')
    axs[i, 3].axvline(x=vline, color='red', linestyle='--')
    ax2 = axs[i, 3].twinx()
    norm_pos_x = gaze_positions_df['norm_pos_x'][gaze_start_idx:gaze_end_idx]
    ax2.scatter(gaze_time, norm_pos_x, label='Gaze norm_pos_x', color='blue', alpha=0.5)
    ax2.set_ylabel('Gaze norm_pos_x')
    ax2.set_ylim(0.25, 0.75)
    # ax2.set_yticks(YTICKS)
    

    # Set common labels
    for j in range(len(column_labels)):
        axs[i, j].set_xlabel('Time (s)')

axs[5, 0].set_ylabel('Mean Brake Pressure Across 4 Wheels [Bar]', fontsize = 16)
axs[5, 1].set_ylabel('center of gravity x-velocity [Km/h]', fontsize = 16)
axs[5, 2].set_ylabel('center of gravity y-acceleration [m/(s^2)]', fontsize = 16)
axs[5, 3].set_ylabel('center of gravity yaw rate [deg/s]', fontsize = 16)

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()